In [ ]:
# Question 2
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read brain image
brain_img = cv2.imread('E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/brain_proton_density_slice.png', cv2.IMREAD_GRAYSCALE)

# Create window/level transformations
def create_window_lut(center, width):
    """Create a window/level LUT for medical image enhancement"""
    lut = np.zeros(256, dtype=np.uint8)
    low = center - width//2
    high = center + width//2
    
    for i in range(256):
        if i < low:
            lut[i] = 0
        elif i > high:
            lut[i] = 255
        else:
            lut[i] = int(255 * (i - low) / (high - low))
    return lut

# Create transformations
lut_white = create_window_lut(center=140, width=80)
lut_gray = create_window_lut(center=110, width=100)

# Apply transformations
white_matter = cv2.LUT(brain_img, lut_white)
gray_matter = cv2.LUT(brain_img, lut_gray)

# COMPREHENSIVE COMPARISON
plt.figure(figsize=(18, 12))

# 1. Original and transformed images
plt.subplot(3, 4, 1)
plt.imshow(brain_img, cmap='gray')
plt.title('Original Brain Image', fontweight='bold')
plt.axis('off')

plt.subplot(3, 4, 2)
plt.imshow(white_matter, cmap='gray')
plt.title('White Matter Enhanced', fontweight='bold')
plt.axis('off')

plt.subplot(3, 4, 3)
plt.imshow(gray_matter, cmap='gray')
plt.title('Gray Matter Enhanced', fontweight='bold')
plt.axis('off')

# 2. Transformation curves
plt.subplot(3, 4, 4)
plt.plot(lut_white, 'b-', linewidth=2, label='White Matter')
plt.plot(lut_gray, 'g-', linewidth=2, label='Gray Matter')
plt.title('Transformation Curves Comparison', fontweight='bold')
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.grid(True, alpha=0.3)
plt.legend()

# 3. Original histogram
plt.subplot(3, 4, 5); plt.hist(brain_img.ravel(), bins=50, color='black', alpha=0.7)
plt.title('Original Histogram', fontweight='bold')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 4. White matter histogram
plt.subplot(3, 4, 6); plt.hist(white_matter.ravel(), bins=50, color='blue', alpha=0.7)
plt.title('White Matter Histogram', fontweight='bold')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 5. Gray matter histogram
plt.subplot(3, 4, 7); plt.hist(gray_matter.ravel(), bins=50, color='green', alpha=0.7)
plt.title('Gray Matter Histogram', fontweight='bold')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 6. Comparison: All histograms
plt.subplot(3, 4, 8)
plt.hist(brain_img.ravel(), bins=50, color='black', alpha=0.3, label='Original')
plt.hist(white_matter.ravel(), bins=50, color='blue', alpha=0.3, label='White Matter')
plt.hist(gray_matter.ravel(), bins=50, color='green', alpha=0.3, label='Gray Matter')
plt.title('All Histograms Comparison', fontweight='bold')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

# 7. Intensity windows visualization
plt.subplot(3, 4, 9)
x = np.arange(256)
plt.plot(x, x, 'k--', alpha=0.5, label='Identity')
plt.plot(lut_white, 'b-', label='White Matter LUT')
plt.plot(lut_gray, 'g-', label='Gray Matter LUT')
plt.axvspan(100, 180, alpha=0.2, color='blue', label='White Matter Window')
plt.axvspan(60, 160, alpha=0.2, color='green', label='Gray Matter Window')
plt.title('Intensity Windows', fontweight='bold')
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# STATISTICAL ANALYSIS
print("=" * 60)
print("BRAIN MATTER ENHANCEMENT - STATISTICAL ANALYSIS")
print("=" * 60)

print(f"\n{'METRIC':<25} {'Original':<12} {'White Matter':<12} {'Gray Matter':<12}")
print("-" * 60)

metrics = {
    'Mean Intensity': [np.mean(brain_img), np.mean(white_matter), np.mean(gray_matter)],
    'Std Deviation': [np.std(brain_img), np.std(white_matter), np.std(gray_matter)],
    'Min Intensity': [np.min(brain_img), np.min(white_matter), np.min(gray_matter)],
    'Max Intensity': [np.max(brain_img), np.max(white_matter), np.max(gray_matter)],
    'Dynamic Range': [np.ptp(brain_img), np.ptp(white_matter), np.ptp(gray_matter)]
}

for metric, values in metrics.items():
    print(f"{metric:<25} {values[0]:<12.2f} {values[1]:<12.2f} {values[2]:<12.2f}")

# TISSUE-SPECIFIC ANALYSIS
def analyze_tissue_range(data, tissue_range, name):
    low, high = tissue_range
    count = np.sum((data >= low) & (data <= high))
    percentage = (count / data.size) * 100
    return percentage

print(f"\n{'TISSUE DISTRIBUTION ANALYSIS':<40} {'Original':<10} {'White Matter':<12} {'Gray Matter':<12}")
print("-" * 70)

tissues = {
    'CSF/Ventricles (0-50)': (0, 50),
    'Gray Matter (51-120)': (51, 120),
    'White Matter (121-180)': (121, 180),
    'Bone/Artifacts (181-255)': (181, 255)
}

for tissue, range_val in tissues.items():
    orig_pct = analyze_tissue_range(brain_img, range_val, tissue)
    white_pct = analyze_tissue_range(white_matter, range_val, tissue)
    gray_pct = analyze_tissue_range(gray_matter, range_val, tissue)
    print(f"{tissue:<40} {orig_pct:<10.1f}% {white_pct:<12.1f}% {gray_pct:<12.1f}%")

print("\n" + "=" * 60)
print("TRANSFORMATION EFFECT ANALYSIS")
print("=" * 60)
print("WHITE MATTER ENHANCEMENT:")
print("- Focuses on intensities 100-180 (brighter tissues)")
print("- Compresses values outside this range to black/white")
print("- Increases contrast within white matter regions")
print("- Makes white matter structures more visible")

print("\nGRAY MATTER ENHANCEMENT:")
print("- Focuses on intensities 60-160 (mid-range tissues)")
print("- Compresses values outside this range to black/white")
print("- Increases contrast within gray matter regions")
print("- Makes gray matter structures more visible")

print("\nWINDOW/LEVEL TRANSFORMATION:")
print("- Medical imaging standard for tissue emphasis")
print("- Center = middle of the intensity window")
print("- Width = range of intensities to enhance")
print("- Values outside window are clipped for maximum contrast")